In [1]:
import pandas as pd
import json

input_ = pd.read_excel('./3.item.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != json.dumps('')].reset_index(drop=True)
input_['No'] = input_['No'].astype(int)

input_

,No,Item Number,Title,Subtitle,Price,Available,Sold,Vehicle W/o Engine,Vehicle W/ Engine,Picture,Url,Description Url,Json_Compatibility,Json_Src,Json_Specific
0,1,295813156245,4Pcs Front Air Suspension Strut Shocks Compres...,,US $572.28,10 available,,,,,https://www.ebay.com/itm/295813156245?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/295813156...,"{""market"": ""US"", ""page"": 2, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/mB8AAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
1,2,275924852464,3PCS Front Air Suspension Shock + Compressor F...,,US $496.79,3 available,,,,,https://www.ebay.com/itm/275924852464?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/275924852...,"{""market"": ""US"", ""page"": 2, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/FiwAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
2,3,275310017049,AC Condenser &amp; Compressor Cooling Kit For ...,,US $216.99,More than 10 available,2 sold,,,,https://www.ebay.com/itm/275310017049?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/275310017...,"{""market"": ""US"", ""page"": 4, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/h3MAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
3,4,276021471698,AC Condenser &amp; AC Compressor Cooling Kit F...,,US $211.99,3 available,,,,,https://www.ebay.com/itm/276021471698?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/276021471...,"{""market"": ""US"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://www.cdnclouds.net/np/8mFkymmx7t...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
4,5,295917179761,AC Condenser &amp; AC Compressor Cooling Kit F...,,US $209.99,More than 10 available,2 sold,,,,https://www.ebay.com/itm/295917179761?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/295917179...,"{""market"": ""US"", ""page"": 3, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/nkwAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,788,276495125650,AC A/C Compressor Clutch For 2003-2011 Mercede...,,US $37.61,More than 10 available,,,,,https://www.ebay.com/itm/276495125650?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/276495125...,"{""market"": ""US"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/p0cAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
786,789,285857563343,AC A/C Compressor Clutch Kits Fits 2004-2006 F...,,US $36.85,More than 10 available,,,,,https://www.ebay.com/itm/285857563343?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/285857563...,"{""market"": ""US"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/WO8AAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
787,790,276465639466,For 2004-2005 2006 Scion xA xB1.5L AC A/C Com...,,US $36.85,More than 10 available,,,,,https://www.ebay.com/itm/276465639466?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/276465639...,"{""market"": ""US"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/5KAAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."
788,791,276465908175,For Toyota Yaris 2007 2008 2009-2012 1.5L 2022...,,US $36.85,More than 10 available,,,,,https://www.ebay.com/itm/276465908175?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/276465908...,"{""market"": ""US"", ""page"": 3, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/kr8AAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,..."


In [2]:
from tqdm import tqdm

output = pd.DataFrame()
for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
    json_ = json.loads(input_.loc[i, 'Json_Compatibility'])
    if json_['market'] == 'US':
        list_vehicle_url = [f'''https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'DE':
        list_vehicle_url = [f'''https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'UK':
        list_vehicle_url = [f'''https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'AU':
        list_vehicle_url = [f'''https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    df_temp = pd.DataFrame({'No': input_.loc[i, 'No'],
                            'Item Number': input_.loc[i, 'Item Number'],
                            'Market': json_['market'],
                            'Page': [j+1 for j in range(json_['page'])],
                            'JOIN_INP': [f'''{input_.loc[i, 'Item Number']};{j+1}''' for j in range(json_['page'])],
                            'Vehicle Url': list_vehicle_url,
                            'Data': json.dumps(json_['data'])})

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')

output.to_excel(f'4.item_2.xlsx', index=False)

output

Progress: 100%|███████████████████████████| 790/790 [00:01<00:00, 482.60it/s]


,No,Item Number,Market,Page,JOIN_INP,Vehicle Url,Data
0,1,295813156245,US,1,295813156245;1,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
1,1,295813156245,US,2,295813156245;2,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
2,2,275924852464,US,1,275924852464;1,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
3,2,275924852464,US,2,275924852464;2,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4,3,275310017049,US,1,275310017049;1,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
...,...,...,...,...,...,...,...
4444,791,276465908175,US,2,276465908175;2,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4445,791,276465908175,US,3,276465908175;3,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4446,792,295541445177,US,1,295541445177;1,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4447,792,295541445177,US,2,295541445177;2,https://www.ebay.com/g/api/finders?module_grou...,"{""scopedContext"": {""catalogDetails"": {""categor..."
